In [3]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np

# 1. 데이터 가져오기 (전체 기간)
name = 'TSLA'
ticker = yf.Ticker(name)
# ADX 계산의 정확성을 위해 첫 번째 폴드 시작일(2018-02-27)보다 넉넉하게 앞선 날짜부터 가져옵니다.
df = ticker.history(start='2017-01-01', end='2025-10-30', interval='1d', auto_adjust=True)

# 2. ADX 지표 계산 (전체 데이터 기준)
# 기간: 14일 (표준)
df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

# 3. TSLA 결과 로그 기반 폴드별 기간 정의
folds = [
    # Fold 1: 2018-02-27 ~ 2019-09-09
    {'label': 'Fold 1', 'start': '2018-02-27', 'end': '2019-09-09'},
    
    # Fold 2: 2019-09-10 ~ 2021-03-22
    {'label': 'Fold 2', 'start': '2019-09-10', 'end': '2021-03-22'},
    
    # Fold 3: 2021-03-23 ~ 2022-09-30
    {'label': 'Fold 3', 'start': '2021-03-23', 'end': '2022-09-30'},
    
    # Fold 4: 2022-10-03 ~ 2024-04-16 (주말/공휴일 등 고려 로그상의 Start 적용)
    {'label': 'Fold 4', 'start': '2022-10-03', 'end': '2024-04-16'},
    
    # Fold 5: 2024-04-17 ~ 2025-10-29
    {'label': 'Fold 5', 'start': '2024-04-17', 'end': '2025-10-29'},
]

# 4. 폴드별 평균 ADX 계산 및 출력
print(f"[{name} 폴드별 14일 ADX 평균]")
print("-" * 55)

adx_results = []

for fold in folds:
    # 해당 기간 데이터 슬라이싱 (Start ~ End)
    mask = (df.index >= fold['start']) & (df.index <= fold['end'])
    fold_data = df.loc[mask]
    
    # 평균 ADX 계산
    avg_adx = fold_data['ADX'].mean()
    adx_results.append(avg_adx)
    
    print(f"{fold['label']} ({fold['start']} ~ {fold['end']}): {avg_adx:.2f}")

print("-" * 55)
print(f"전체 폴드 평균 ADX: {np.mean(adx_results):.2f}")

[TSLA 폴드별 14일 ADX 평균]
-------------------------------------------------------
Fold 1 (2018-02-27 ~ 2019-09-09): 21.71
Fold 2 (2019-09-10 ~ 2021-03-22): 31.77
Fold 3 (2021-03-23 ~ 2022-09-30): 22.09
Fold 4 (2022-10-03 ~ 2024-04-16): 27.09
Fold 5 (2024-04-17 ~ 2025-10-29): 25.38
-------------------------------------------------------
전체 폴드 평균 ADX: 25.61
